# Face Area Calculations

## Overview

This notebook will showcase the different area calculation options provided by UXarray
the grid topology attributes and variables stored in the `UXarray.Grid`
object.

For more details on how to load in data, check out our [previous usage
example](https://uxarray.readthedocs.io/en/latest/examples/read-grid-data.html)

**This notebook has the following sections:**
1. Compute total area of all faces
2. Options for accuracy of area calculation functions
3. Getting area of all the faces (not total)
4. Calculate area of a single triangle
5. Calculate area of multiple vertices in spherical coordinates


## 1. Load mesh files and calculate total area of all faces

We will be using the `outCSne30.ug` grid file which is encoded in the UGRID convention.
Let us first read in the data:

In [81]:
import uxarray as ux
import numpy as np
import xarray as xr

In [82]:
# Base data path
base_path = "../../test/meshfiles/"

# Path to Grid files
ugrid_path = base_path + "/ugrid/outCSne30/outCSne30.ug"

# Load grid files and create UXarray Grid objects
ugrid_ds = xr.open_dataset(ugrid_path)

ugrid = ux.Grid(ugrid_ds)

Now call the function `calculate_total_face_area()` to calculate the total

In [83]:
# Calculate total area of the first (ugrid) mesh
area = ugrid.calculate_total_face_area()
area
# 4 Pi is the area 4*3.14 = 12.56

12.566370614678554

## 2. Options for `Grid.calculate_total_face_area` functions
### Quadrature Rule: Gaussian or Triangular
### Order: 1 to 10 for Gaussian, 1, 4, 8, 10 and 12 for triangular
### Default: Triangular and order 4

In [84]:
# Notice that the area is different from the first mesh above
# Now we use "triangular" quadrature rule and order 1
# This is less time consuming as the order is only 1
ugrid.calculate_total_face_area(quadrature_rule="triangular", order=1)

12.566370614678554

## 3. Getting area of all the faces (not total)

In [85]:
# calling the method again will use the cached value
ugrid.face_areas

array([0.00211174, 0.00211221, 0.00210723, ..., 0.00210723, 0.00211221,
       0.00211174])

# Now calculate the area again with "compute face_areas" function
# pass arguments: quadrature_rule "gaussian" and order 4

In [86]:
all_face_areas = ugrid.compute_face_areas(quadrature_rule="gaussian", order=4)
all_face_areas.sum()

12.566370614678554

## 4. Calculate area of a single triangle (cartesian coordinates)

In [87]:
# define a new grid with 3 vertices
# Assume the units in meters - this is a big triangle
verts = [[[0.57735027, -5.77350269e-01, -0.57735027],
         [0.57735027, 5.77350269e-01, -0.57735027],
         [-0.57735027, 5.77350269e-01, -0.57735027]]]
# Note: initialization using array of vertices

# load our vertices into a UXarray Grid object
vgrid = ux.Grid(verts, vertices=True,
                        islatlon=False,
                        concave=False)

# Note the incorrect units for the x and y coordinates (expand the cell Mesh2_node_x etc. to see)
vgrid.ds

<xarray.Dataset>
Dimensions:           (nMesh2_node: 3, nMesh2_face: 1, nMaxMesh2_face_nodes: 3)
Dimensions without coordinates: nMesh2_node, nMesh2_face, nMaxMesh2_face_nodes
Data variables:
    Mesh2             float64 nan
    Mesh2_node_x      (nMesh2_node) float64 -0.5774 0.5774 0.5774
    Mesh2_node_y      (nMesh2_node) float64 0.5774 -0.5774 0.5774
    Mesh2_node_z      (nMesh2_node) float64 -0.5774 -0.5774 -0.5774
    Mesh2_face_nodes  (nMesh2_face, nMaxMesh2_face_nodes) uint64 1 2 0

In [88]:
# vgrid.calculate_total_face_area()
# This in incorrect as the units are not in meters

In [89]:
# Set correct units for the x and y coordinates
vgrid.Mesh2_node_x.attrs["units"] = "m"
vgrid.Mesh2_node_y.attrs["units"] = "m"
vgrid.Mesh2_node_z.attrs["units"] = "m"

# It is important to set the units for the x and y coordinates
# otherwise the area will be incorrect, as the units are assumed to be spherical by default
# Without the above setting the coordinates will be assumed to be in spherical coordinates
# and the area will be calculated incorrectly

# Calculate the area of the triangle
area_gaussian = vgrid.calculate_total_face_area(
            quadrature_rule="gaussian", order=5)
area_gaussian

1.0475702709086985

## 5. Calculate area of multiple vertices in spherical coordinates

In [90]:
faces_verts_ndarray = np.array([
    np.array([[150, 10, 0], [160, 20, 0], [150, 30, 0], [135, 30, 0], [125, 20, 0],
              [135, 10, 0]]),
    np.array([[125, 20, 0], [135, 30, 0], [125, 60, 0], [110, 60, 0], [100, 30, 0],
              [105, 20, 0]]),
    np.array([[95, 10, 0], [105, 20, 0], [100, 30, 0], [85, 30, 0], [75, 20, 0],
              [85, 10, 0]]),
])

In [91]:
# load our vertices into a UXarray Grid object
verts_grid = ux.Grid(faces_verts_ndarray, vertices=True,
                        islatlon=True,
                        concave=False)
verts_grid.ds

<xarray.Dataset>
Dimensions:           (nMesh2_node: 14, nMesh2_face: 3, nMaxMesh2_face_nodes: 6)
Dimensions without coordinates: nMesh2_node, nMesh2_face, nMaxMesh2_face_nodes
Data variables:
    Mesh2             float64 nan
    Mesh2_node_x      (nMesh2_node) int64 75 85 85 95 100 ... 135 150 150 160
    Mesh2_node_y      (nMesh2_node) int64 20 10 30 10 30 20 ... 10 30 10 30 20
    Mesh2_node_z      (nMesh2_node) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 0
    Mesh2_face_nodes  (nMesh2_face, nMaxMesh2_face_nodes) uint64 11 13 ... 0 1

In [92]:
verts_grid.face_areas

array([0.14323746, 0.25118746, 0.12141312])